In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)

In [2]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [3]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [4]:
def yunrip_data_prep():
    basedir = './국토교통부_실거래가_공개시스템/연립다세대/매매/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        if '해제사유발생일' in df.columns.tolist():
            df = df.drop(columns=['해제사유발생일'])

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['전용면적단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
    
    concat_df['년'] = concat_df['계약날짜'].dt.year
    
    cols_to_drop = ['시군구', '번지', '본번', '부번', '지번주소', '계약년월', '계약일', '도로명']
    
    concat_df = concat_df[concat_df['년'] >= 2015]
    
    return concat_df[['지번주소'] + [col for col in concat_df.columns if col not in cols_to_drop]]

In [5]:
def officetel_data_prep():
    basedir = './국토교통부_실거래가_공개시스템/오피스텔/매매/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        if '해제사유발생일' in df.columns.tolist():
            df = df.drop(columns=['해제사유발생일'])

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['전용면적단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
    
    concat_df['년'] = concat_df['계약날짜'].dt.year
    
    cols_to_drop = ['시군구', '번지', '본번', '부번', '지번주소', '계약년월', '계약일', '도로명']
    
    concat_df = concat_df[concat_df['년'] >= 2015]
        
    return concat_df[['지번주소'] + [col for col in concat_df.columns if col not in cols_to_drop]]

In [6]:
yunrip_df = yunrip_data_prep()
print(yunrip_df.shape)
yunrip_df.head()

16it [00:01, 11.58it/s]


(329098, 11)


,지번주소,건물명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),년
358389,서울특별시 강남구 개포동 1216-4,(1216-4),33.77,21.67,24800,4,2012.0,3.0,2015-03-26,734.379627,2015
358390,서울특별시 강남구 개포동 1216-4,(1216-4),35.87,23.02,27200,4,2012.0,3.0,2015-06-23,758.293839,2015
358391,서울특별시 강남구 개포동 1216-4,(1216-4),29.12,18.68,22200,4,2012.0,3.0,2015-07-20,762.362637,2015
358392,서울특별시 강남구 개포동 1216-4,(1216-4),29.97,19.23,22500,3,2012.0,3.0,2015-08-06,750.750751,2015
358393,서울특별시 강남구 개포동 170-18,(170-18),26.60,21.53,21000,1,1988.0,27.0,2015-08-21,789.473684,2015


In [7]:
officetel_df = officetel_data_prep()
print(officetel_df.shape)
officetel_df.head()

16it [00:00, 44.93it/s]


(77955, 10)


,지번주소,단지명,전용면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),년
100156,서울특별시 강남구 개포동 13-3,대청타워,43.24,22000,14,1997.0,18.0,2015-01-08,508.788159,2015
100157,서울특별시 강남구 개포동 13-3,대청타워,32.44,15800,21,1997.0,18.0,2015-01-12,487.053021,2015
100158,서울특별시 강남구 개포동 13-3,대청타워,32.44,16000,10,1997.0,18.0,2015-01-19,493.218249,2015
100159,서울특별시 강남구 개포동 13-3,대청타워,32.44,15400,21,1997.0,18.0,2015-01-26,474.722565,2015
100160,서울특별시 강남구 개포동 13-3,대청타워,31.91,16000,26,1997.0,18.0,2015-01-28,501.410216,2015


In [8]:
land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_4.csv')
print(land_specs_df.shape)
land_specs_df.head()

(8706295, 11)


,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 100-0,2013,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,325000.0
1,서울특별시 강남구 개포동 100-0,2014,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,330000.0
2,서울특별시 강남구 개포동 100-0,2015,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,335000.0
3,서울특별시 강남구 개포동 100-0,2016,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,345000.0
4,서울특별시 강남구 개포동 100-0,2017,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,355000.0


In [9]:
land_specs_df['도로접면'].value_counts()

세로한면(가)    2688474
지정되지않음     1289408
세로한면(불)    1008484
세로각지(가)     726127
소로한면        627975
소로각지        503622
광대소각        428623
중로각지        393928
중로한면        298912
광대로한면       278183
맹지          185238
광대세각        171918
세로각지(불)     105268
Name: 도로접면, dtype: int64

In [10]:
yunrip_merge_df = yunrip_df.merge(land_specs_df, on=['지번주소', '년'])
print(yunrip_merge_df.shape)
yunrip_merge_df.head()

(326789, 20)


,지번주소,건물명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 1216-4,(1216-4),33.77,21.67,24800,4,2012.0,3.0,2015-03-26,734.379627,2015,대,257.0,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),3470000.0
1,서울특별시 강남구 개포동 1216-4,(1216-4),35.87,23.02,27200,4,2012.0,3.0,2015-06-23,758.293839,2015,대,257.0,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),3470000.0
2,서울특별시 강남구 개포동 1216-4,(1216-4),29.12,18.68,22200,4,2012.0,3.0,2015-07-20,762.362637,2015,대,257.0,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),3470000.0
3,서울특별시 강남구 개포동 1216-4,(1216-4),29.97,19.23,22500,3,2012.0,3.0,2015-08-06,750.750751,2015,대,257.0,제2종일반주거지역,지정되지않음,다세대,평지,정방형,세로각지(가),3470000.0
4,서울특별시 강남구 개포동 170-18,(170-18),26.60,21.53,21000,1,1988.0,27.0,2015-08-21,789.473684,2015,대,130.3,제2종일반주거지역,지정되지않음,다세대,평지,세로장방,세로한면(가),4370000.0


In [11]:
officetel_merge_df = officetel_df.merge(land_specs_df, on=['지번주소', '년'])
print(officetel_merge_df.shape)
officetel_merge_df.head()

(75617, 19)


,지번주소,단지명,전용면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 13-3,대청타워,43.24,22000,14,1997.0,18.0,2015-01-08,508.788159,2015,대,4188.5,일반상업지역,지정되지않음,업무용,평지,세로장방,광대소각,12670000.0
1,서울특별시 강남구 개포동 13-3,대청타워,32.44,15800,21,1997.0,18.0,2015-01-12,487.053021,2015,대,4188.5,일반상업지역,지정되지않음,업무용,평지,세로장방,광대소각,12670000.0
2,서울특별시 강남구 개포동 13-3,대청타워,32.44,16000,10,1997.0,18.0,2015-01-19,493.218249,2015,대,4188.5,일반상업지역,지정되지않음,업무용,평지,세로장방,광대소각,12670000.0
3,서울특별시 강남구 개포동 13-3,대청타워,32.44,15400,21,1997.0,18.0,2015-01-26,474.722565,2015,대,4188.5,일반상업지역,지정되지않음,업무용,평지,세로장방,광대소각,12670000.0
4,서울특별시 강남구 개포동 13-3,대청타워,31.91,16000,26,1997.0,18.0,2015-01-28,501.410216,2015,대,4188.5,일반상업지역,지정되지않음,업무용,평지,세로장방,광대소각,12670000.0


In [12]:
reference_df = pd.read_excel('./감정평가사_자료/이승준_총괄표.xlsx', header=[0,1]).dropna(subset=[('소재지','소재지')]).sort_values([('소재지', '지역'),('소재지', '소재지'), ('소재지', '지번')]).reset_index(drop=True)
print(reference_df.shape)
reference_df.head()

(131, 39)


소재지                토지내용                                           \
    지역  소재지       지번  용도지역 도로너비 토지면적(㎡)        토지매매금액 토지매매단가\n(원/㎡)   
0  강남구  개포동  1195-10  2종일주  6미터   265.7  2.970000e+09    11178020.0   
1  강남구  개포동   1199-7  2종일주  4미터   248.0  2.550000e+09    10282258.0   
2  강남구  개포동   1216-7  2종일주  8미터   258.1  2.931000e+09    11356064.0   
3  강남구  역삼동    707-9  일반상업  8미터   363.5  9.500000e+09    26134801.0   
4  강남구  역삼동    751-6  2종일주  6미터   391.8  5.900000e+09    15058703.0   

                       건물내용                                                   \
  공시지가와격차\n(개공/매매)     건물용도  공급면적(㎡)  전유면적(㎡)    전용율 용적율\n(%) 건폐율\n(%) 구분건물호   
0            0.433    다세대주택   519.54   432.22  0.832   195.54    59.62  15개호   
1            0.365    다세대주택   463.84   390.67  0.842   187.04    59.61  14개호   
2            0.472  도시행생활주택   489.46   400.29  0.818   199.62    59.89  15개호   
3            0.580  도시행생활주택  1695.85  1289.62  0.760   199.62    59.89  15개호   
4            0.441  도시행생활주택   658.88   537.65  0.816   168.17    46.35  20개호   

             수입/비용                                                        \
  건물동수  건물층수   의뢰인        기준시점        전체평가금액        투입비용합계           순이익   
0  1개동  지상5층  SH공사  2020.09.23  4.719500e+09  4.167814e+09  5.516858e+08   
1  1개동  지상5층  LH공사  2019.11.29  4.040000e+09  3.593249e+09  4.467511e+08   
2  1개동  지상5층  SH공사  2019.11.29  4.586000e+09  4.054965e+09  5.310348e+08   
3  1개동  지상5층  LH공사  2019.11.29  2.219983e+10  1.398811e+10  8.211721e+09   
4  1개동  지상6층  LH공사  2019.11.29  9.810500e+09  7.604511e+09  2.205989e+09   

         평가단가(원/전유㎡)                 매출원가(원/전유㎡)                   사업진행일자  \
    순이익율         전유면적          공급면적        전유면적기준      공급면적기준      토지계약일자   
0  0.117   10919208.0  9.083997e+06  9.642807e+06   8372395.0  2019.02.26   
1  0.111   10341209.0  8.709524e+06  9.197658e+06   8074624.0  2018.08.21   
2  0.116   11456694.0  9.369509e+06  1.013007e+07   8312283.0  2019.01.24   
3  0.370   17214239.0  1.115945e+07  1.084669e+07  16602582.0  2019.08.14   
4  0.225   18247001.0  1.488966e+07  1.414398e+07  12107552.0  2019.01.04   

                                               건축비\n(원/평)    원가비율          \
         허가일자        착공일자       사용승인일 Unnamed: 32_level_1  토지원가비율  건축원가비율   
0  2019.06.18  2019.06.18  2020.01.13           5500000.0  0.7563  0.2139   
1  2018.12.31  2019.01.07  2019.06.04           5400000.0  0.7520  0.2175   
2  2019.06.17  2019.07.23  2020.02.25           5400000.0  0.7667  0.2034   
3  2019.07.15  2020.01.03  2020.10.07           5800000.0  0.7189  0.2568   
4  2019.03.05  2019.04.20  2019.11.29           5500000.0  0.8249  0.1486   

                                        비고  
     부대비용    금융비용   합계 Unnamed: 38_level_1  
0  0.0060  0.0239  1.0                 NaN  
1  0.0061  0.0244  1.0                 NaN  
2  0.0057  0.0242  1.0                 NaN  
3  0.0072  0.0170  1.0                 NaN  
4  0.0042  0.0223  1.0                 NaN

In [13]:
reference_df[('소재지', '지번')] = reference_df[('소재지', '지번')].apply(landnum_modifier)

In [14]:
reference_df['지번주소'] = '서울특별시 ' + reference_df[('소재지', '지역')] + ' ' + reference_df[('소재지', '소재지')]\
+ ' ' + reference_df[('소재지', '지번')]

In [15]:
def date_modifier(x):
    # x is a string or datetime
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if '-' in x:
            x = x.replace('-', '.')
            
        splitted = x.split('.')
        
        if len(splitted) == 1:
            return np.nan
        else:
            return x

In [16]:
reference_df['기준시점2'] = reference_df[('수입/비용', '기준시점')].apply(date_modifier)

In [17]:
def get_year(x):
    if pd.isna(x) == True:
        return x
    else:
        return int(x.split('.')[0])

In [18]:
reference_df['년'] = reference_df['기준시점2'].apply(get_year)

In [19]:
reference_df['년'] = reference_df['년'].fillna(2020)
reference_df['년'].isna().sum()

0

In [20]:
def get_bd_type(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if '오피' in x:
            return '오피스텔'
        else:
            return '다세대'

In [21]:
reference_df['건물유형'] = reference_df[('건물내용', '건물용도')].apply(get_bd_type)

In [22]:
reference_df['최근3년50분위'] = np.nan
reference_df['최근3년50분위_평가단가와의격차'] = np.nan
reference_df['최근3년평균'] = np.nan
reference_df['최근3년평균_평가단가와의격차'] = np.nan
reference_df['최근2년50분위'] = np.nan
reference_df['최근2년50분위_평가단가와의격차'] = np.nan
reference_df['최근2년평균'] = np.nan
reference_df['최근2년평균_평가단가와의격차'] = np.nan
reference_df['당해50분위'] = np.nan
reference_df['당해50분위_평가단가와의격차'] = np.nan
reference_df['당해평균'] = np.nan
reference_df['당해평균_평가단가와의격차'] = np.nan

In [23]:
for i in tqdm(range(reference_df.shape[0])):
    gu = reference_df.loc[i, ('소재지', '지역')].replace(' ', '')
    dong = reference_df.loc[i, ('소재지', '소재지')].replace(' ', '')
    
    
    land_purpose = reference_df.loc[i, ('토지내용', '용도지역')].replace(' ', '')[:3]
    year = int(reference_df.loc[i, '년'])
    bd_type = reference_df.loc[i, '건물유형'][0]
    
    if bd_type == '오피스텔':
        target_df = officetel_merge_df[(officetel_merge_df['지번주소'].str.contains(gu+' '+dong))&
                                       (officetel_merge_df['용도지역명1'].str.contains(land_purpose))&
                                       (officetel_merge_df['건물연식'] <= 5)
                                      ]
    else:
        target_df = yunrip_merge_df[(yunrip_merge_df['지번주소'].str.contains(gu+' '+dong))&
                                    (yunrip_merge_df['용도지역명1'].str.contains(land_purpose))&
                                    (yunrip_merge_df['건물연식'] <= 5)
                                   ]    
    
    
    target_year_minus_two_df = target_df[(target_df['년'] <= year)&(target_df['년'] >= year-2)]
    target_year_minus_one_df = target_df[(target_df['년'] <= year)&(target_df['년'] >= year-1)]
    target_year_df = target_df[target_df['년'] == year]
        
        
    reference_df.loc[i, '최근3년50분위'] = target_year_minus_two_df['전용면적단가(만원/㎡)'].median()
    reference_df.loc[i, '최근3년평균'] = target_year_minus_two_df['전용면적단가(만원/㎡)'].mean()
    reference_df.loc[i, '최근2년50분위'] = target_year_minus_one_df['전용면적단가(만원/㎡)'].median()
    reference_df.loc[i, '최근2년평균'] = target_year_minus_one_df['전용면적단가(만원/㎡)'].mean()
    reference_df.loc[i, '당해50분위'] = target_year_df['전용면적단가(만원/㎡)'].median()
    reference_df.loc[i, '당해평균'] = target_year_df['전용면적단가(만원/㎡)'].mean()

100%|████████████████████████████████████████████████████████████████████████████████| 131/131 [00:22<00:00,  5.74it/s]


In [24]:
cols_list = []
for i in range(reference_df.shape[1]):
    if i < 39:
        cols_list.append(reference_df.columns[i][0] + '_' + reference_df.columns[i][1])
    else:
        cols_list.append(reference_df.columns[i][0])

In [25]:
reference_df.columns = cols_list

In [26]:
cols = ['지번주소', '기준시점2', '건물내용_건물용도', '건물유형', '토지내용_용도지역', '토지내용_도로너비',
        '평가단가(원/전유㎡) _전유면적', '최근3년50분위', '최근3년50분위_평가단가와의격차', '최근3년평균',
        '최근3년평균_평가단가와의격차', '최근2년50분위', '최근2년50분위_평가단가와의격차', '최근2년평균',
        '최근2년평균_평가단가와의격차', '당해50분위', '당해50분위_평가단가와의격차', '당해평균', '당해평균_평가단가와의격차'
       ]
selected_df = reference_df[cols]

In [27]:
selected_df['평가단가(원/전유㎡) _전유면적'] = selected_df['평가단가(원/전유㎡) _전유면적'] / 10000

In [28]:
selected_cols = ['최근3년50분위', '최근3년평균', '최근2년50분위', '최근2년평균', '당해50분위', '당해평균']
for col in selected_cols:
    selected_df[col+'_평가단가와의격차'] = selected_df[col] / selected_df['평가단가(원/전유㎡) _전유면적']
    selected_df[col+'_오차절대값'] = np.abs(selected_df[col] - selected_df['평가단가(원/전유㎡) _전유면적'])
    selected_df[col+'_절대값오차율'] = selected_df[col+'_오차절대값'] / selected_df['평가단가(원/전유㎡) _전유면적'] * 100

In [29]:
base_cols = ['지번주소', '기준시점2', '건물내용_건물용도', '건물유형', '토지내용_용도지역', '토지내용_도로너비',
             '평가단가(원/전유㎡) _전유면적'
            ]
cols_to_show = []
for col in selected_cols:
    cols_to_show.append(col)
    cols_to_show.append(col+'_절대값오차율')

In [30]:
selected2_df = selected_df[(selected_df['건물내용_건물용도'].str.contains('다세'))|(selected_df['건물내용_건물용도'].str.contains('오피'))].dropna(subset=['기준시점2'])
selected2_df = selected2_df[base_cols + cols_to_show]
print(selected2_df.shape)
selected2_df

(52, 19)


,지번주소,기준시점2,건물내용_건물용도,건물유형,토지내용_용도지역,토지내용_도로너비,평가단가(원/전유㎡) _전유면적,최근3년50분위,최근3년50분위_절대값오차율,최근3년평균,최근3년평균_절대값오차율,최근2년50분위,최근2년50분위_절대값오차율,최근2년평균,최근2년평균_절대값오차율,당해50분위,당해50분위_절대값오차율,당해평균,당해평균_절대값오차율
0,서울특별시 강남구 개포동 1195-10,2020.09.23,다세대주택,다세대,2종일주,6미터,1091.9208,1079.676961,1.121312,1065.218106,2.445479,1114.612052,2.078104,1115.573296,2.166137,1146.667354,5.013784,1138.218150,4.239992
1,서울특별시 강남구 개포동 1199-7,2019.11.29,다세대주택,다세대,2종일주,4미터,1034.1209,988.660278,4.396065,986.346819,4.619777,1005.361930,2.781007,1007.860927,2.539352,1060.929350,2.592390,1071.290025,3.594273
5,서울특별시 강남구 역삼동 778-15,2019.11.29,도생-다세대,다세대,2종일주,6미터 및\n 4미터,1656.9493,1227.524205,25.916610,1245.563652,24.827896,1261.127596,23.888583,1308.884122,21.006387,1420.128276,14.292593,1438.316844,13.194879
7,서울특별시 강동구 고덕동 292-5,2019.06.14,다세대주택,다세대,2종일반주거,4미터,989.5452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,서울특별시 강동구 고덕동 292-6,2019.06.14,다세대주택,다세대,2종일주,4미터,988.1178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,서울특별시 강동구 길동 140-0,2019.06.14,단지형다세대,다세대,3종일주,6미터,750.4237,882.549216,17.606789,857.707494,14.296429,892.991711,18.998335,967.756881,28.961396,904.295403,20.504643,1026.342379,36.768386
13,서울특별시 강동구 길동 96-4,2019.06.14,다세대-도시형주택,다세대,3종일주,4미터,746.9423,882.549216,18.154939,857.707494,14.829150,892.991711,19.552971,967.756881,29.562468,904.295403,21.066300,1026.342379,37.405845
14,서울특별시 강동구 명일동 350-1외,2020.11.11,도생(단지형다세대),다세대,3종일주,8미터,713.2919,703.605981,1.357918,696.136712,2.405073,722.416009,1.279155,709.635601,0.512595,729.400749,2.258381,714.925793,0.229064
16,서울특별시 강동구 암사동 433-69외,2020.10.23,다세대주택,다세대,3종일주,4미터,711.0120,999.983951,40.642345,941.888515,32.471536,950.297751,33.654249,865.062565,21.666381,779.827379,9.678512,779.827379,9.678512
17,서울특별시 강동구 암사동 454-26,2019.04.01,다세대주택,다세대,2종일주,4미터,609.4015,670.774088,10.070961,708.792854,16.309667,686.684362,12.681764,730.540084,19.878288,686.537173,12.657611,729.806237,19.757867


In [31]:
for col in selected2_df.columns:
    if '_절대값오차율' in col:
        print(col)
        print(selected2_df[col].mean())

최근3년50분위_절대값오차율
8.950185988242657
최근3년평균_절대값오차율
8.409585004461293
최근2년50분위_절대값오차율
7.561520059198589
최근2년평균_절대값오차율
8.148235568682795
당해50분위_절대값오차율
7.0265144908271
당해평균_절대값오차율
8.684020835375197
